 First, let's setup all of this...

In [1]:
from pathlib import Path
import spotipy
import json
import sys
from pprint import pprint
import pandas as pd

# User configuration
credential_fp = Path(r'private/spotify_credential.json')

# List of interesting scopes
scope = ' '.join([
    'user-library-read',
    'user-top-read',
])


In [2]:
with open(credential_fp) as file:
    credential = json.load(file)

# if 'token' in credential.keys() and scope == credential['token']['scope']:
#     print(f'token already available for following scope:\n\t{scope.split(" ")}')
# else:
try:
    token_code = spotipy.util.prompt_for_user_token(
        username=credential["username"],
        client_id=credential["client_id"],
        client_secret=credential["client_secret"],
        redirect_uri="http://localhost/",
        scope=scope,
        cache_path=Path(f'private/.cache-{credential["username"]}')
    )
    credential["token"] = {'code':token_code,
                            'scope':scope}
    with open(credential_fp, 'w') as file:
        json.dump(credential, file)
except:
    print("error with token retrieval")
    raise


 Now we start

In [3]:
sp = spotipy.Spotify(auth=credential['token']['code'])



In [4]:
save_tracks_count = 501
limit = 50

result_top_tracks = sp.current_user_top_tracks(limit=limit)
result_top_artists = sp.current_user_top_artists(limit=limit)

result_liked_songs = None
for offset in range(0, save_tracks_count, limit):
    limit_temp = min([limit, save_tracks_count-offset])
    result_temp = sp.current_user_saved_tracks(limit=limit_temp, offset=offset)
    if result_liked_songs == None:
        result_liked_songs = result_temp
    else:
        result_liked_songs['items'] = result_liked_songs['items'] + result_temp['items']


In [5]:
liked_songs = result_liked_songs['items']
top_artists = result_top_artists['items']
top_tracks = result_top_tracks['items']



In [6]:
pprint(top_tracks[0].keys())
pprint(top_artists[0].keys())
pprint(top_tracks[0])



dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])
dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])
{'album': {'album_type': 'ALBUM',
           'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/25CgccXK9hJBTuaMF44zeR'},
                        'href': 'https://api.spotify.com/v1/artists/25CgccXK9hJBTuaMF44zeR',
                        'id': '25CgccXK9hJBTuaMF44zeR',
                        'name': 'The Spirit of the Beehive',
                        'type': 'artist',
                        'uri': 'spotify:artist:25CgccXK9hJBTuaMF44zeR'}],
           'available_markets': ['AD',
                                 'AE',
                                 'AR',
                                 'AT',
                                 'AU'

In [8]:
df_lt = pd.DataFrame(liked_songs)
df_lt.head()



,added_at,track
0,2020-02-05T00:04:18Z,"{'album': {'album_type': 'single', 'artists': ..."
1,2020-02-04T23:25:33Z,"{'album': {'album_type': 'single', 'artists': ..."
2,2020-02-04T22:55:57Z,"{'album': {'album_type': 'single', 'artists': ..."
3,2020-02-04T22:48:29Z,"{'album': {'album_type': 'single', 'artists': ..."
4,2020-02-04T22:37:06Z,"{'album': {'album_type': 'single', 'artists': ..."
